In [1]:
import matplotlib.pyplot as plt

import pandas as pd
import gensim
import numpy as np
from gensim.models import Word2Vec
from time import time  # To time our operations
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

data = pd.read_json("preprocessingDoneRight.json")

C:\Users\gneur\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
sentences = data[data["year"] == 2016]["modded1"].tolist()
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[i]) for i, _d in enumerate(sentences)]
tagged_data

[TaggedDocument(words=['foxnews', 'live', 'members', 'family', '1150', 'pm', 'ring', 'new', 'year', 'together', 'make', 'america', 'great'], tags=[0]),
 TaggedDocument(words=['happy', 'new', 'year', 'amp', 'thank'], tags=[1]),
 TaggedDocument(words=['happy', 'new', 'year', 'thank', 'great', 'family', 'support'], tags=[2]),
 TaggedDocument(words=['jallenaip', 'hillary', 'said', 'fog', 'war', 'explanation', 'lies', 'benghazi', 'fog', 'allowed', 'wh', 'vote', 'trump', 'potus'], tags=[3]),
 TaggedDocument(words=['sprinklermanus', 'cnn', 'realdonaldtrump', 'theyre', 'spending', 'millions', 'youre', 'still', 'going', 'win', 'go', 'donald', 'trump'], tags=[4]),
 TaggedDocument(words=['well', 'year', 'officially', 'begun', 'many', 'stops', 'planned', 'working', 'hard', 'win', 'turn', 'country', 'around'], tags=[5]),
 TaggedDocument(words=['marie7777777777', 'realdonaldtrump', 'love', 'u', 'trump', 'family', 'god', 'bless'], tags=[6]),
 TaggedDocument(words=['jodil792', 'standing', 'spreading',

In [3]:
model = Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(tagged_data)
%time model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)


Wall time: 18.4 s


In [4]:
ranks = []
second_ranks = []
for doc_id in range(len(tagged_data)):
    inferred_vector = model.infer_vector(tagged_data[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

import collections

collections.Counter(ranks)  # Results vary between runs due to random seeding and very small corpus


C:\Users\gneur\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Counter({0: 2024,
         2: 220,
         8: 51,
         4: 104,
         7: 44,
         109: 2,
         1: 477,
         80: 5,
         22: 12,
         3: 156,
         33: 9,
         11: 37,
         39: 7,
         28: 15,
         9: 50,
         176: 1,
         34: 8,
         5: 84,
         154: 1,
         15: 22,
         23: 18,
         1474: 1,
         1074: 1,
         45: 7,
         726: 1,
         24: 18,
         10: 37,
         54: 7,
         18: 16,
         53: 2,
         32: 11,
         104: 2,
         41: 4,
         16: 19,
         36: 8,
         13: 33,
         19: 11,
         42: 7,
         6: 58,
         139: 1,
         25: 17,
         52: 5,
         146: 3,
         1044: 1,
         12: 22,
         703: 1,
         31: 8,
         88: 6,
         62: 4,
         134: 3,
         37: 8,
         29: 7,
         385: 1,
         17: 26,
         63: 6,
         2257: 1,
         102: 2,
         56: 4,
         1267: 1,
         1594:

In [5]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(tagged_data[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]: 
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(tagged_data[sims[index][0]].words)))


Document (4208): «rt realdonaldtrump happy birthday donaldjtrumpjr»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d300,n5,w5,mc2,s0.001,t3):

MOST (3407, 0.9687870740890503): «rt gop timkaines words debates2016»

SECOND-MOST (4208, 0.9517670273780823): «rt realdonaldtrump happy birthday donaldjtrumpjr»

MEDIAN (2940, 0.5051911473274231): «soon calling mr brexit»

LEAST (1160, -0.7422667145729065): «clinton sanders people disrupted rally chicago say must talk people phony politicians»



In [6]:
X = []
for i in range(0,len(tagged_data)):
    X.append( model.infer_vector(tagged_data[i].words))

In [19]:
from sklearn.cluster import Birch
from sklearn.metrics import silhouette_score

k = 5
brc = Birch(branching_factor=200, n_clusters=k, threshold=0.5, compute_labels=True)
brc.fit(X)

clusters = brc.predict(X)

labels = brc.labels_

#print ("Clusters: ")
#print (clusters)
m = silhouette_score(X, labels, metric="cosine")

print ("Silhouette_score: ")
print (k, m)

Silhouette_score: 
5 0.060172867


# HIERARCHICAL

In [30]:
#silhouette looks promising but resulting clusters shitty

from sklearn.cluster import AgglomerativeClustering

k=5

for k in range(2,10):

    hier = AgglomerativeClustering(affinity='cosine', linkage='average', n_clusters=k)
    hier.fit(X)

    labels = hier.labels_

    print(k, silhouette_score(X,labels,metric="cosine"))

2 0.2542728
3 0.18674654
4 0.1387985
5 0.12833458
6 0.123553716
7 0.08432452
8 0.06562981
9 0.07275988


In [37]:
hier = AgglomerativeClustering(affinity='cosine', linkage='average', n_clusters=5)
hier.fit(X)

labels = hier.labels_

print(silhouette_score(X,labels,metric="cosine"))

data1 = data[data["year"] == 2016]
data1["labels"] = labels

0.12833458


C:\Users\gneur\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [38]:
clusters_wordCount = []
for cluster in range(0,5):
    c = data1[data1["labels"] == cluster]
   # words = pd.Series(' '.join(c['m']).split()).value_counts()
    words = pd.Series(' '.join(c['modded1']).split()).value_counts()
    m = words.mean()
    words = words[words > m]
    clusters_wordCount.append(words)
    print("Cluster " + str(cluster) + " dimension " + str(len(c)))

for c in range(0,5):
    print("cluster ", c , clusters_wordCount[c][0:10])

Cluster 0 dimension 37
Cluster 1 dimension 4129
Cluster 2 dimension 25
Cluster 3 dimension 17
Cluster 4 dimension 1
cluster  0 hillary     25
crooked     18
people      10
clinton      9
bernie       6
cnn          6
great        5
carolina     5
sanders      5
today        5
dtype: int64
cluster  1 thank                    634
great                    591
hillary                  434
trump                    433
realdonaldtrump          380
amp                      363
trump2016                346
america                  285
clinton                  276
makeamericagreatagain    272
dtype: int64
cluster  2 jobs       10
america     6
hillary     6
back        5
rubio       4
country     4
great       4
way         4
ohio        4
soon        3
dtype: int64
cluster  3 realdonaldtrump    7
nytimes            5
media              5
amazing            5
people             4
great              4
amp                4
thank              4
trump2016          3
us                 3
dtype: int6